# Designing a Python library for building prototypes around MinHash

This is very much work-in-progress. May be the software and or ideas presented with be the subject of a peer-reviewed or self-published write-up. For now the URL for this is: https://github.com/lgautier/mashing-pumpkins

MinHash in the context of biological sequenced was introduced by the Maryland Bioinformatics Lab [add reference here].

Building a MinHash is akin to taking a sample of all k-mers / n-grams found in a sequence and using that sample as a signature or sketch for that sequence.

## A look at convenience *vs* performance

Moving Python code to C leads to performance improvement... sometimes.

### Test sequence

First we need a test sequence. Generating a random one quickly can be achieved as follows, for example. If you already have you own way to generate a sequence, or your own benchmark sequence, the following code cell can be changed so as to end up with a variable `sequence` that is a `bytes` object containing it.

In [1]:
# we take a DNA sequence as an example, but this is arbitrary and not necessary.
alphabet = b'ATGC'
# create a lookup structure to go from byte to 4-mer
# (a arbitrary byte is a bitpacked 4-mer)
quad = [None,]*(len(alphabet)**4)
i = 0
for b1 in alphabet:
    for b2 in alphabet:
        for b3 in alphabet:
            for b4 in alphabet:
                quad[i] = bytes((b1,b2,b3,b4))
                i += 1
# random bytes for a 1M genome (order of magnitude for a bacterial genome)
import ssl
size = int(1E6)
sequencebitpacked = ssl.RAND_bytes(int(size/4))
sequence = bytearray(int(size))
for i, b in zip(range(0, len(sequence), 4), sequencebitpacked):
    sequence[i:(i+4)] = quad[b]
sequence = bytes(sequence)

### Kicking the tires with `sourmash`

The executable `sourmash` is a nice package from the dib-lab implemented in Python and including a library [add reference here]. Perfect for trying out quick what MinHash sketches can do.

We will create a MinHash of maximum size 1000 (1000 elements) and of k-mer size 21 (all ngrams of length 21 across the input sequences will be considered for inclusion in the MinHash. At the time of writing MinHash is implemented in C/C++ and use that as a reference for speed, as we measure the time it takes to process our 1M reference sequence

In [2]:
from sourmash_lib._minhash import MinHash

import timeit

stmt = """
smh = MinHash(1000, 21)
smh.add_sequence(sequence)
"""
t_sourmash = timeit.timeit(stmt,
                           setup='from __main__ import sequence, MinHash; sequence=sequence.decode("utf-8")',
                           number=5)
print("%.2f seconds" % t_sourmash)

1.55 seconds


This is awesome. The sketch for a bacteria-sized DNA sequence can be computed very quickly (about a second on my laptop).

### Redisigning it all for convenience and flexibility

We have redesigned what a class could look like, and implemented that design in Python
foremost for our own convenience and to match the claim of convenience. Now how bad is the impact on performance ?

Our new design allows flexibility with respect to the hash function used, and to initially illustrate our point we use `mmh` an existing Python package wrapping MurmurHash3, the hashing function used in `MASH` and `sourmash`.

In [3]:
# make a hashing function to match our design
import mmh3
def hashfun(sequence, nsize, hbuffer, w=100):
    n = min(len(hbuffer), len(sequence)-nsize+1)
    for i in range(n):
        ngram = sequence[i:(i+nsize)]
        hbuffer[i] = mmh3.hash64(ngram)[0]
    return n

from mashingpumpkins.minhashsketch import MaxHashNgramSketch

stmt = 'mhs = MaxHashNgramSketch(21, 1000, hashfun); mhs.add(sequence, hashtype="q")'
t_basic = timeit.timeit(stmt,
                        setup='from __main__ import MaxHashNgramSketch, hashfun, sequence',
                        number=5)
print("%.2f seconds" % t_basic)

print("Our Python implementation is %.2f times slower." % (t_basic / t_sourmash))

2.03 seconds
Our Python implementation is 1.31 times slower.


Ah. Our Python implementation only using `mmh3` and the standard library is not slower.

**note: ** the careful reader will not that we have a MaxHash rather than a MinHash. This is so to use algorithms in the Python stan

There is more to it though. The code in "mashingpumpkins" is doing more by keeping track of the k-mer/n-gram along with the hash value in order to allow the generation of inter-operable sketch [add reference to discussion on GitHub].

We can modifying our class to stop storing the associated k-mer (only keep the hash value) to see if it improves performances:

In [4]:
class NoNgrams(MaxHashNgramSketch):
    
    def _make_elt(self, h, ngram):
        return (h, )

stmt = 'mhs = NoNgrams(21, 1000, hashfun); mhs.add(sequence, hashtype="q")'
t_nongrams = timeit.timeit(stmt,
                           setup='from __main__ import NoNgrams, hashfun, sequence',
                           number=5)

print("%.2f seconds" % t_nongrams)

print("Our Python implementation is %.2f times slower." % (t_nongrams / t_sourmash))

1.98 seconds
Our Python implementation is 1.28 times slower.


No real difference here. Storing the k-mers / n-grams only has an impact on the added memory required to stored the 1,000
k-mers of length 21.

Our design in computing batches of hash values each time C is reached for MurmurHash3. We have implemented the small C function require to call MurmurHash for several k-mers, and when using it we have interesting performance gains:

In [5]:
from mashingpumpkins._murmurhash3 import hasharray
hashfun = hasharray
stmt = 'mhs = MaxHashNgramSketch(21, 1000, hashfun); mhs.add(sequence)'
t_batch = timeit.timeit(stmt,
                        setup='from __main__ import MaxHashNgramSketch, hashfun, sequence',
                        number=5)

print("%.2f seconds" % t_batch)

print("Our Python implementation is %.2f times faster." % (t_sourmash / t_batch))


0.42 seconds
Our Python implementation is 3.69 times faster.


Wow!

At the time of writing this is approximatively 3 times faster than C-implemented `sourmash`. And we are doing more work (we are keeping the ngrams / kmers associated with hash values).

We also have an alternative fast hashing available (XXHash). Slightly faster than MurmurHash3.

In [ ]:
from mashingpumpkins import _xxhash
hashfun = _xxhash.hasharray
stmt = 'mhs = MaxHashNgramSketch(21, 1000, hashfun); mhs.add(sequence)'
t_batch_xxh = timeit.timeit(stmt,
                            setup='from __main__ import MaxHashNgramSketch, hashfun, sequence',
                            number=5)

print("%.2f seconds" % t_batch_xxh)

print("Our Python implementation is %.2f times faster." % (t_sourmash / t_batch_xxh))

0.43 seconds
Our Python implementation is 3.57 times faster.


## To infinite and beyond

Now how much time should it take to compute signature for various references ?

First we check quickly that the time is roughly proportional to the size of the reference:

In [ ]:
# random genome between 3 and 10 times larger than before

for size in (int(3E6), int(5E6), int(1E7), int(2E7)):
    print('\n* size: {:,}'.format(size))
    sequencebitpacked = ssl.RAND_bytes(int(size/4))
    sequence5 = bytearray(int(size))
    for i, b in zip(range(0, len(sequence5), 4), sequencebitpacked):
        sequence5[i:(i+4)] = quad[b]
    sequence5 = bytes(sequence5)

    stmt = """
smh = MinHash(1000, 21)
smh.add_sequence(sequence)
    """
    t_sourmash_5 = timeit.timeit(stmt,
                                 setup='from __main__ import sequence5, MinHash; sequence=sequence5.decode("utf-8")',
                                 number=5)
    print("%.2f seconds" % t_sourmash_5)
    print("size ratio: %.2f - time ratio: %.2f" % (len(sequence5)/len(sequence), t_sourmash_5/t_sourmash))

    hashfun = hasharray
    stmt = 'mhs = MaxHashNgramSketch(21, 1000, hashfun); mhs.add(sequence)'
    t_batch_5 = timeit.timeit(stmt,
                              setup='from __main__ import MaxHashNgramSketch, hashfun,'
                                    'sequence5; sequence=sequence5',
                              number=5)
    print("%.2f seconds" % t_batch_5)
    print("size ratio: %.2f - time ratio: %.2f" % (len(sequence5)/len(sequence), t_batch_5/t_batch))
    print("Our Python implementation is %.2f times faster." % (t_sourmash_5 / t_batch_5))


* size: 3,000,000
4.87 seconds
size ratio: 3.00 - time ratio: 3.15
1.12 seconds
size ratio: 3.00 - time ratio: 2.69
Our Python implementation is 4.33 times faster.

* size: 5,000,000
8.10 seconds
size ratio: 5.00 - time ratio: 5.24
1.86 seconds
size ratio: 5.00 - time ratio: 4.44
Our Python implementation is 4.36 times faster.

* size: 10,000,000
16.96 seconds
size ratio: 10.00 - time ratio: 10.97


The time spent appears proportional to the size of the input sequence within than range of sizes... with the suspicion that our code is scaling better with increasing sequence size (relatively faster as the size is increasing).

### Back-of-envelope time

This would allow us to compute the signatures for >1,800 bacterial genomes per hour and per CPU. On a laptop 
(where this notebook was initially run). Using the 2 cores, this would be >3,600 bacterial genomes per hour.

When considering raw reads from a sequencing run, this would put the 5E11 bases optimally out of a latest
Illumina MiSeq sequencer processed on 4 core in about 13 hours. 7 hours on 8 cores. Not great, but quite bearable
when an 8-core machine can be rented per hour on cloud for cheap... and this is still in Python so customization
and ideas can be implemented rarther easily and quickly.